In [1]:
import requests
from difflib import SequenceMatcher
from datetime import datetime

class ChatBot:
    def __init__(self, name, responses, api_key):
        self.name = name
        self.responses = responses
        self.api_key = api_key
    
    @staticmethod
    def calculate_similarity(input_sentence, response_sentence):
        sequence = SequenceMatcher(a=input_sentence, b=response_sentence)
        return sequence.ratio()
    
    def get_best_response(self, user_input):
        highest_similarity = 0.0
        best_match = 'Sorry, I didn\'t understand that.'
        
        for response, reply in self.responses.items():
            similarity = self.calculate_similarity(user_input, response)
            if similarity > highest_similarity:
                highest_similarity = similarity
                best_match = reply
        return best_match, highest_similarity
    
    def get_weather(self, city):
        url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={self.api_key}&units=metric"
        response = requests.get(url)
        data = response.json()
        
        if data["cod"] != 200:
            return "Sorry, I couldn't fetch the weather for that location."
        
        weather = data["weather"][0]["description"]
        temp = data["main"]["temp"]
        return f"The weather in {city} is {weather} with a temperature of {temp}°C."
    
    def run(self):
        print(f'Hello! My name is {self.name}. How can I help you today?')
        
        while True:
            user_input = input('You: ')
            # Check if the user asked for weather
            if 'weather in' in user_input.lower():
                city = user_input.split('weather in')[-1].strip()
                response = self.get_weather(city)
                print(f'{self.name}: {response}')
                continue
            
            # Check for the best response based on similarity
            response, similarity = self.get_best_response(user_input)
            
            # If similarity is too low, provide default response
            if similarity < 0.50:
                response = 'Sorry, I didn\'t understand that. Can you ask something else?'
            
            # Special command to get time
            if response == 'GET_TIME':
                response = f'The time is: {datetime.now():%H:%M}'
            
            # Special command to exit
            elif 'exit' in user_input.lower():
                print("Goodbye! Have a great day!")
                break
            
            print(f'{self.name}: {response} (Similarity: {similarity:.2f})')

def main():
    api_key = "YOUR_API_KEY"  # Replace with your OpenWeatherMap API key
    responses = {
        'hello': 'Hello! How are you today?',
        'how are you': 'I am doing well, thanks for asking!',
        'good morning': 'Good morning! How can I assist you today?',
        'what time is it': 'GET_TIME',
        'time': 'GET_TIME',
        'bye': 'Goodbye! Have a great day!',
        'weather in': 'GET_WEATHER'
    }
    
    chatbot = ChatBot(name='Kinshu', responses=responses, api_key=api_key)
    chatbot.run()

if __name__ == '__main__':
    main()

Hello! My name is Kinshu. How can I help you today?
You: time
Kinshu: Sorry, I didn't understand that. Can you ask something else? (Similarity: 0.42)
You: time is it
Kinshu: The time is: 12:24 (Similarity: 0.80)
You: exit
Goodbye! Have a great day!
